<a href="https://www.kaggle.com/code/ibkya12/map-0-97-leashmp-competition?scriptVersionId=180590949" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 37.0 MB/s eta 0:00:00


In [2]:
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 64.1 MB/s eta 0:00:00


## Data Preparation

The training and testing data paths are defined for the .parquet files. We use duckdb to scan search through the large training sets. Just to get started lets sample out an equal number of positive and negatives. 

This query selects an equal number of samples where binds equals 0 (non-binding) and 1 (binding), limited to 30,000 each, to avoid model bias towards a particular class.

In [3]:
import duckdb
import pandas as pd

train_path = '/kaggle/input/leash-predict-chemical-bindings/train.parquet'
test_path = '/kaggle/input/leash-predict-chemical-bindings/test.parquet'

con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 70000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 70000)""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,60149173,Cc1cccc(NC(=O)OCC2c3ccccc3-c3ccccc32)c1C(=O)O,COCc1ccccc1CN,COC(=O)c1cc(N)cc(F)c1,COCc1ccccc1CNc1nc(Nc2cc(F)cc(C(=O)OC)c2)nc(Nc2...,HSA,0
1,127634836,O=C(N[C@H](Cc1csc2ccccc12)C(=O)O)OCC1c2ccccc2-...,COC(=O)c1cc(F)c(F)cc1N,CC1CC(CN)C(C)O1,COC(=O)c1cc(F)c(F)cc1Nc1nc(NCC2CC(C)OC2C)nc(N[...,HSA,0
2,160948950,O=C(Nc1cc(Br)ccc1C(=O)O)OCC1c2ccccc2-c2ccccc21,NCc1c[nH]c2ccccc12,C[Si](C)(C)C#Cc1ccc(N)cn1,C[Si](C)(C)C#Cc1ccc(Nc2nc(NCc3c[nH]c4ccccc34)n...,BRD4,0
3,108673802,O=C(N[C@@H](Cc1csc2ccccc12)C(=O)O)OCC1c2ccccc2...,NCc1ccccn1,CC(O)(CN)c1ccsc1,CC(O)(CNc1nc(NCc2ccccn2)nc(N[C@@H](Cc2csc3cccc...,sEH,0
4,279308521,O=C(O)[C@@H]1CCCCN1C(=O)OCC1c2ccccc2-c2ccccc21,Cc1cccc(OCCCN)c1C.Cl,N#Cc1ccc(N)cc1,Cc1cccc(OCCCNc2nc(Nc3ccc(C#N)cc3)nc(N3CCCC[C@H...,HSA,0


## Feature Preprocessing

Lets grab the smiles for the fully assembled molecule `molecule_smiles` and generate ecfps for it. We could choose different radiuses or bits, but 2 and 1024 is pretty standard.

In [5]:
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder

# Convert SMILES to RDKit molecules
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)

# Generate ECFPs
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

df['ecfp'] = df['molecule'].apply(generate_ecfp)

## Train Model

In [6]:
# One-hot encode the protein_name
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

# Combine ECFPs and one-hot encoded protein_name
X = [ecfp + protein for ecfp, protein in zip(df['ecfp'].tolist(), protein_onehot.tolist())]
y = df['binds'].tolist()

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,molecule,ecfp
0,60149173,Cc1cccc(NC(=O)OCC2c3ccccc3-c3ccccc32)c1C(=O)O,COCc1ccccc1CN,COC(=O)c1cc(N)cc(F)c1,COCc1ccccc1CNc1nc(Nc2cc(F)cc(C(=O)OC)c2)nc(Nc2...,HSA,0,<rdkit.Chem.rdchem.Mol object at 0x7d7853b8ed50>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,127634836,O=C(N[C@H](Cc1csc2ccccc12)C(=O)O)OCC1c2ccccc2-...,COC(=O)c1cc(F)c(F)cc1N,CC1CC(CN)C(C)O1,COC(=O)c1cc(F)c(F)cc1Nc1nc(NCC2CC(C)OC2C)nc(N[...,HSA,0,<rdkit.Chem.rdchem.Mol object at 0x7d78538d77d0>,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,160948950,O=C(Nc1cc(Br)ccc1C(=O)O)OCC1c2ccccc2-c2ccccc21,NCc1c[nH]c2ccccc12,C[Si](C)(C)C#Cc1ccc(N)cn1,C[Si](C)(C)C#Cc1ccc(Nc2nc(NCc3c[nH]c4ccccc34)n...,BRD4,0,<rdkit.Chem.rdchem.Mol object at 0x7d78538d7840>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,108673802,O=C(N[C@@H](Cc1csc2ccccc12)C(=O)O)OCC1c2ccccc2...,NCc1ccccn1,CC(O)(CN)c1ccsc1,CC(O)(CNc1nc(NCc2ccccn2)nc(N[C@@H](Cc2csc3cccc...,sEH,0,<rdkit.Chem.rdchem.Mol object at 0x7d78538d78b0>,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."
4,279308521,O=C(O)[C@@H]1CCCCN1C(=O)OCC1c2ccccc2-c2ccccc21,Cc1cccc(OCCCN)c1C.Cl,N#Cc1ccc(N)cc1,Cc1cccc(OCCCNc2nc(Nc3ccc(C#N)cc3)nc(N3CCCC[C@H...,HSA,0,<rdkit.Chem.rdchem.Mol object at 0x7d78538d7920>,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."


In [8]:
from xgboost import XGBClassifier

In [9]:
xgb = XGBClassifier()

xgb_model = xgb.fit(X_train , y_train)

y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.2f}")

Mean Average Precision (mAP): 0.97


# Best Score received with CatBoostClassifier

In [10]:
from catboost import CatBoostClassifier
catb_model = CatBoostClassifier().fit(X_train, y_train)
y_pred_proba = catb_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.2f}")

Learning rate set to 0.077259
0:	learn: 0.6662083	total: 151ms	remaining: 2m 31s
1:	learn: 0.6428459	total: 235ms	remaining: 1m 57s
2:	learn: 0.6194990	total: 310ms	remaining: 1m 43s
3:	learn: 0.6034943	total: 391ms	remaining: 1m 37s
4:	learn: 0.5897107	total: 464ms	remaining: 1m 32s
5:	learn: 0.5756531	total: 540ms	remaining: 1m 29s
6:	learn: 0.5632134	total: 622ms	remaining: 1m 28s
7:	learn: 0.5534126	total: 701ms	remaining: 1m 26s
8:	learn: 0.5456474	total: 779ms	remaining: 1m 25s
9:	learn: 0.5393062	total: 855ms	remaining: 1m 24s
10:	learn: 0.5330576	total: 926ms	remaining: 1m 23s
11:	learn: 0.5272586	total: 997ms	remaining: 1m 22s
12:	learn: 0.5211662	total: 1.08s	remaining: 1m 21s
13:	learn: 0.5158795	total: 1.16s	remaining: 1m 21s
14:	learn: 0.5114996	total: 1.23s	remaining: 1m 20s
15:	learn: 0.5069831	total: 1.3s	remaining: 1m 20s
16:	learn: 0.5032361	total: 1.38s	remaining: 1m 19s
17:	learn: 0.4999541	total: 1.44s	remaining: 1m 18s
18:	learn: 0.4974604	total: 1.51s	remaining: 

Look at that Average Precision score. We did amazing! 

Actually no, we just overfit. This is likely recurring theme for this competition. It is easy to predict molecules that come from the same corner of chemical space, but generalizing to new molecules is extremely difficult.

## Test Prediction

 The trained Random Forest model is then used to predict the binding probabilities. These predictions are saved to a CSV file, which serves as the submission file for the Kaggle competition.

In [11]:
import os

# Process the test.parquet file chunk by chunk
test_file = '/kaggle/input/leash-predict-chemical-bindings/test.csv'
output_file = 'submission.csv'  # Specify the path and filename for the output file

# Read the test.parquet file into a pandas DataFrame
for df_test in pd.read_csv(test_file, chunksize=100000):

    # Generate ECFPs for the molecule_smiles
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)

    # One-hot encode the protein_name
    protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))

    # Combine ECFPs and one-hot encoded protein_name
    X_test = [ecfp + protein for ecfp, protein in zip(df_test['ecfp'].tolist(), protein_onehot.tolist())]

    # Predict the probabilities
    probabilities = catb_model.predict_proba(X_test)[:, 1]

    # Create a DataFrame with 'id' and 'probability' columns
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})

    # Save the output DataFrame to a CSV file
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))
